In [2]:
import cobra
from os.path import join
    
# Load a cobra model from anywhere you like
data_dir = "/Users/lizrad/Documents/Vibrio_folder/"
model=cobra.io.read_sbml_model(join(data_dir, "iVnat_newbigg.xml"))



In [ ]:
# generate a json file to load into Escher
cobra.io.save_json_model(model, 'iVnat_newbigg.json')


In [ ]:
from escher import Builder
b = Builder(map_name='e_coli_core.Core metabolism')
b.display_in_browser()

In [ ]:
maps = Builder(model=model, map_name='e_coli_core.Core metabolism')

maps.display_in_browser()

In [ ]:
model_modified = model.copy()
# for example, delete a reaction
#model_modified.reactions.GAPD.delete()

In [3]:
data_dir = "/Users/lizrad/Documents/Vibrio_folder/"
ecoli_model=cobra.io.read_sbml_model(join(data_dir, "iML1515.xml"))
ecoli_model

Name,iML1515
Memory address,0x010f2176a0
Number of metabolites,1877
Number of reactions,2712
Objective expression,-1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685 + 1.0*BIOMASS_Ec_iML1515_core_75p37M
Compartments,"cytosol, extracellular space, periplasm"


In [10]:
ecoli_met_in_vibrio=[]
for e_met in ecoli_model.metabolites:

    
    for met in model.metabolites:
        if "bigg.metabolite" not in met.annotation.keys():
            continue
        try:
            bigg, compartment=e_met.id.split("_", -1)
            
            if bigg in met.annotation["bigg.metabolite"] and met.compartment[0]==compartment:
                print(met.annotation["bigg.metabolite"])
                
                print(compartment,met.compartment)
                #print("ecoli:", bigg, compartment)
                #print("vibrio:", met.id, met.compartment)
                met.id=e_met.id
                print(met.id)
                model.repair()
        except:
            pass

2pg
c c0
2pg_c
2pglyc
c c0
13dpg
c c0
13dpg_c
prbamp
c c0
prbamp_c
grdp
c c0
grdp_c
gg4abut
c c0
gg4abut_c
3ohexACP
c c0
3ohexACP_c
dcamp
c c0
dca_c
hdca
c c0
ocdca
c c0
ddcacoa
c c0
pep
c c0
pep_c
h
c c0
h_c
dhpt
c c0
gthrd
c c0
2mhop
c c0
thmpp
c c0
2mpdhl
c c0
hxan
c c0
nadh
c c0
gluside_cho
c c0
ichor
c c0
23ddhb
c c0
fadh2
c c0
4hpro_LT
c c0
2ahethmpp
c c0
3hhdcoa
c c0
3ohdcoa
c c0
3hanthrn
c c0
h2o2
c c0
h2
c c0
q8h2
c c0
hepdp
c c0
cenchddd
c c0
dhcinnm
c c0
ahcys
c c0
rhcys
c c0
hcys__L
c c0
dhptd
c c0
10fthf
c c0
anth
c c0
5hxkynam
c c0
dhna
c c0
hisp
c c0
hspmd
c c0
6hmhpt
c c0
nadph
c c0
h2mb4p
c c0
tdchola
c c0
gt1a_cho
c c0
3hoctACP
c c0
dhap
c c0
cdp4dh6doglc
c c0
hmgth
c c0
Sfglutth
c c0
etha
c c0
ala_L_Thr__L
c c0
4hbald
c c0
4hbz
c c0
2mahmp
c c0
thmmp
c c0
5mthf
c c0
mlthf
c c0
mhpglu
c c0
hpglu
c c0
adphep_LD
c c0
hlipa
c c0
dha
c c0
dhlpro
c c0
phe__L
c c0
3ohexACP
c c0
his__L
c c0
cdpdhdec9eg
c c0
3c3hmp
c c0
2ameph
c c0
3hpimcoa
c c0
h2s
c c0
pphn
c c0
2mhob
c c0


c c0
kdo2lipid4
c c0
kdolipid4
c c0
argsuc
c c0
argsuc_c
ichor
c c0
chor
c c0
chor_c
thymd
c c0
thymd_c
dgdp
c c0
dgdp_c
glu1sa
c c0
glu1sa_c
cit
e e0
cit_e
fe3dcit
e e0
q8h2
c c0
q8h2_c
fe2
e e0
fe2_e
galt
e e0
galt_e
5prdmbz
c c0
5prdmbz_c
cynt
e e0
cynt_e
ps160
c c0
ps160_c
dhna
c c0
dhna_c
6pgc
c c0
6pgc_c
udpgal
c c0
udpgal_c
udpgalfur
c c0
['nh3', 'nh4']
c c0
nh4_c
rml1p
c c0
rml1p_c
cytd
c c0
cytd_c
thymd
c c0
thym
c c0
thym_c
sbzcoa
c c0
sbzcoa_c
galt
e e0
gal_e
acnam
c c0
acnam_c
['aacald', 'amacald']
c c0
aacald_c
hhlipa
c c0
hhlipa_c
phhlipa
c c0
hphhlipa
c c0
phphhlipa
c c0
kphphhlipa
c c0
['HC02172', 'zn2']
c c0
zn2_c
3odecACP
c c0
3odecACP_c
1tdecg3p
c c0
1tdecg3p_c
pa140
c c0
pa140_c
cdpdtdec7eg
c c0
cdpdtdec7eg_c
2aobut
c c0
2aobut_c
4r5au
c c0
4r5au_c
gam
e e0
gam_e
acgam
e e0
4hbz
c c0
4hbz_c
['dscl', 'shcl']
c c0
dscl_c
phthr
c c0
phthr_c
n8aspmd
c c0
n8aspmd_c
ditp
c c0
ditp_c
hg2
c c0
hg2_c
2tpr3dpcoa
c c0
2tpr3dpcoa_c
atp
c c0
atp_c
datp
c c0
prbatp
c c0
indole
c 

3odcoa
c c0
3odcoa_c
ugmda
c c0
ugmda_c
['nh3', 'nh4']
e e0
nh4_e
idp
c c0
idp_c
didp
c c0
ahdt
c c0
ahdt_c
hdca
e e0
hdca_e
cenchddd
c c0
cenchddd_c
tmrs2eACP
c c0
tmrs2eACP_c
pe180
c c0
pe180_c
2agpe180
c c0
2hdec9eg3p
c c0
2hdec9eg3p_c
cl
c c0
cl_c
mucl
c c0
glyclt
c c0
g6p_B
c c0
2ddg6p
c c0
tag6p__D
c c0
g6p
c c0
g6p_c
trdrd
c c0
trdrd_c
mnl1p
c c0
mnl1p_c
uagmda
c c0
uaagmda
c c0
agm
c c0
agm_c
prpp
c c0
prpp_c
histd
c c0
histd_c
orot5p
c c0
orot
c c0
orot_c
damp
c c0
damp_c
fmn
c c0
fmn_c
3oddcoa
c c0
3oddcoa_c
trnaglu
c c0
trnaglu_c
ins
e e0
ins_e
q8
c c0
q8_c
q8h2
c c0
2dmmq8
c c0
['orn', 'orn__L']
c c0
orn_c
sucorn
c c0
acorn
c c0
orn__D
c c0
etha
c c0
etha_c
ethamp
c c0
betald
c c0
betald_c
hdca
c c0
hdca_c
fgam
c c0
fgam_c
['dd2coa', 'trans_dd2coa']
c c0
dd2coa_c
hdd2coa
c c0
iasp
c c0
iasp_c
ppal
c c0
ppal_c
pphn
c c0
pphn_c
prlp
c c0
prlp_c
2mahmp
c c0
2mahmp_c
nicrnt
c c0
nicrnt_c
no2
e e0
o2
e e0
o2_e
co2
e e0
succ
e e0
succ_e
mql8
c c0
mql8_c
N1aspmd
c c0
N1aspmd_c
met

In [26]:
for e_reac in ecoli_model.reactions:

    
    for reac in model.reactions:
        if "bigg.reaction" not in reac.annotation.keys():
            continue
        try:
            bigg, compartment=e_reac.id.split("_", -1)
            
            if e_reac.id in reac.annotation["bigg.reaction"]: #and reac.compartment[0]==compartment:
                print("ecoli id and vibrio id:", e_reac.id, reac.annotation["bigg.reaction"])
                if type(met.annotation["bigg.metabolite"]) == list:
                    for bigg_list in met.annotation["bigg.metabolite"]:
                        if bigg_list == e_reac.id:
                            reac.id=e_reac.id
                            print("from list:", reac.id)
                            model.repair()
                else:
                            
                #print(compartment,reac.compartment)
                #print("ecoli:", bigg, compartment)
                #print("vibrio:", reac.id, reac.compartment)
                    reac.id=e_reac.id
                    print("not list:",reac.id)
                    model.repair()
        except:
            pass

ecoli id and vibrio id: PFK_3 PFK_3
not list: PFK_3
ecoli id and vibrio id: SERD_L ['SERD_L', 'SER_AL', 'THRAL']
not list: SERD_L
ecoli id and vibrio id: RBK_L1 RBK_L1
not list: RBK_L1
ecoli id and vibrio id: PFK_2 PFK_2
not list: PFK_2
ecoli id and vibrio id: PSP_L ['PSP', 'PSP_L', 'PSP_Lpp']
not list: PSP_L
ecoli id and vibrio id: ASPt2_3pp ['ASPt2_3', 'ASPt2_3pp']
not list: ASPt2_3pp
ecoli id and vibrio id: L_LACD3 L_LACD3
not list: L_LACD3
ecoli id and vibrio id: L_LACD2 L_LACD2
not list: L_LACD2
ecoli id and vibrio id: PSP_Lpp ['PSP', 'PSP_L', 'PSP_Lpp']
not list: PSP_Lpp
ecoli id and vibrio id: L_LACt2rpp ['L_LACt2r', 'L_LACt2rpp', 'L_LACtm', 'r2516']
ecoli id and vibrio id: SUCCt2_3pp ['SUCCt2_3', 'SUCCt2_3pp']
not list: SUCCt2_3pp
ecoli id and vibrio id: THRD_L ['TAL', 'THRD_L', 'THRD_Lm']
ecoli id and vibrio id: SERD_D SERD_D
not list: SERD_D
ecoli id and vibrio id: LDH_D2 LDH_D2
ecoli id and vibrio id: LDH_D ['LDH_D', 'LDHm']
not list: LDH_D
ecoli id and vibrio id: LDH_D LDH_